### Student Information
Name: 盧子涵

Student ID: 113065542

GitHub ID: luzi2023

Kaggle name: luzi8451

Kaggle private scoreboard snapshot: 

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [ ]:
### Begin Assignment Here

: 

### =======================Second Part=================================

In [1]:
import pandas as pd
import numpy as np
import json
import emoji
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import transformers
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
raw_tweet = "tweets_DM.json"
data = []
with open(raw_tweet, "r") as f_in:
    for line in f_in:
        data.append(json.loads(line))

data_identification = pd.read_csv("./data_identification.csv")
submission = pd.read_csv("./sampleSubmission.csv")
# submission.head(3)

In [3]:
# 定義表情符號到關鍵字的映射字典
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]'
}

# emoji_dict = {
#     '😂': '[emoji]',
#     '❤️': '[emoji]',
#     '😍': '[emoji]',
#     '😭': '[emoji]',
#     '❤': '[emoji]',
#     '😊': '[emoji]',
#     '🙏': '[emoji]',
#     '😘': '[emoji]',
#     '💕': '[emoji]',
#     '🔥': '[emoji]',
#     '😩': '[emoji]',
#     '🤔': '[emoji]',
#     '💯': '[emoji]',
#     '💙': '[emoji]',
#     '🙄': '[emoji]',
#     '😁': '[emoji]',
#     '🙌': '[emoji]',
#     '🙏🏾': '[emoji]',
#     '👍': '[emoji]',
#     '🙏🏽': '[emoji]'
# }

# 定義清理推文文本的函數
def clean_tweet(text, emoji_dict):
    # 將定義的表情符號替換為對應的關鍵詞
    for emj, keyword in emoji_dict.items():
        text = text.replace(emj, keyword)
    # 移除其餘的表情符號
    text = emoji.replace_emoji(text, replace='')
    # 移除 <LH> 標籤
    text = re.sub(r'<LH>', '', text)
    # 移除多餘的空白字元
    text = text.strip()
    return text

In [ ]:
# processed_tweets = []
# for entry in data:
#     # 檢查 '_source' 和 'tweet' 是否存在於記錄中
#     if '_source' in entry and 'tweet' in entry['_source']:
#         tweet = entry['_source']['tweet']
#         # 檢查 'text' 是否存在於 'tweet' 中
#         if 'text' in tweet:
#             tweet_text = tweet['text']
#             cleaned_text = clean_tweet(tweet_text, emoji_dict)
#             # 創建處理後的推文記錄，保留 '_source' 和 'tweet'
#             processed_tweet = {
#                 '_source': {
#                     'tweet': tweet.copy()
#                 }
#             }
#             # 更新清理後的文本
#             processed_tweet['_source']['tweet']['text'] = cleaned_text
#             processed_tweets.append(processed_tweet)
#         else:
#             print("記錄中缺少 'text' 鍵")
#     else:
#         print("記錄中缺少 '_source' 或 'tweet' 鍵")

# # 將處理後的推文資料存儲為 JSON 檔案
# with open('tweets_DM_filtered_1.json', 'w', encoding='utf-8') as outfile:
#     json.dump(processed_tweets, outfile, ensure_ascii=False, indent=4)

In [3]:
processed_tweets = []
with open('tweets_DM_filtered_emoji.json', 'r', encoding='utf-8') as f_in:
  data = json.load(f_in)
  processed_tweets.append(data)

In [4]:
# 將處理後的資料轉換為 DataFrame
df_processed = pd.DataFrame(processed_tweets)

df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})

In [5]:
df = df.merge(data_identification, on='tweet_id', how='left')

In [6]:
emotions = pd.read_csv("emotion.csv")
emotions['emotion'].value_counts()

emotion
joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: count, dtype: int64

In [7]:
train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']
train_data = train_data.merge(emotions, on='tweet_id', how='left')
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

# 統一submission跟test_data的順序
test_data.rename(columns={'tweet_id': 'id', 'hashtags': 'hashtags', 'text':'text', 'identification': 'identification'}, inplace=True)

new_submission = submission.copy()

new_submission = new_submission.merge(test_data, on='id')
new_submission.drop(['hashtags', 'identification'], axis='columns', inplace=True)

y_train_data = train_data['emotion']
X_train_data = train_data['text']
X_test_data = new_submission['text']

# 標籤編碼
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train_data)

/tmp/ipykernel_1816555/2092790352.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.rename(columns={'tweet_id': 'id', 'hashtags': 'hashtags', 'text':'text', 'identification': 'identification'}, inplace=True)


In [ ]:
new_submission.head(3)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train, test_size=0.3, random_state=42, stratify=y_train)

In [10]:
from transformers import BertTokenizer, BertModel, DistilBertTokenizer

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 50
tokenized_train = tokenizer(X_train.tolist(), max_length=max_length, truncation=True, padding=True, return_tensors="pt")
# tokenized_val = tokenizer(X_val.tolist(), max_length=max_length, truncation=True, padding=True, return_tensors="pt")
tokenized_test = tokenizer(X_test_data.tolist(), max_length=max_length, truncation=True, padding=True, return_tensors="pt")

In [ ]:
print(tokenized_train)

In [ ]:
# import torch
# from torch import nn
# from transformers import BertModel, BertTokenizer

# class BertClassifier(nn.Module):
#     def __init__(self):
#         super(BertClassifier, self).__init__()
#         self.bert = BertModel.from_pretrained("bert-base-uncased")
#         # self.bert = BertModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
#         self.classifier = nn.Linear(self.bert.config.hidden_size, 8)

#     def forward(self, input_ids, attention_mask, token_type_ids=None):
#         # BERT 模型輸出
#         outputs = self.bert(input_ids=input_ids, 
#                             attention_mask=attention_mask, 
#                             token_type_ids=token_type_ids
#                             )
#         pooled_output = outputs.pooler_output  # BERT 的 [CLS] token 表示
#         logits = self.classifier(pooled_output)  # 線性層進行分類
#         return logits

In [11]:
import torch
from torch import nn
from transformers import DistilBertModel

class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        # 使用 DistilBERT 模型
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, 8)  # 假設有 8 個分類

    def forward(self, input_ids, attention_mask):
        # DistilBERT 輸出 last_hidden_state
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # 使用 last_hidden_state 的第一個 token 作為句子的表示
        cls_output = outputs.last_hidden_state[:, 0, :]  # 取 [CLS] token 嵌入
        logits = self.classifier(cls_output)  # 線性層進行分類
        return logits


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertClassifier().to(device)

input_ids = torch.tensor(tokenized_train['input_ids'].numpy()).to(device)
attention_mask = torch.tensor(tokenized_train['attention_mask'].numpy()).to(device)
# token_type_ids = torch.tensor(tokenized_train['token_type_ids'].numpy()).to(device)

train_input = {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    # 'token_type_ids': token_type_ids,
}

y_train = torch.tensor(y_train).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [14]:
epochs = 5
batch_size = 16
dataset = torch.utils.data.TensorDataset(
    train_input['input_ids'], train_input['attention_mask'], y_train
)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for batch in tqdm(dataloader):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        
        # 前向傳播
        logits = model(input_ids, attention_mask)
        
        # 計算損失
        loss = criterion(logits, labels)
        total_loss += loss.item()
        
        # 計算準確率
        _, preds = torch.max(logits, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_samples += labels.size(0)
        
        # 反向傳播和更新權重
        loss.backward()
        optimizer.step()
    
    # 每個 epoch 結束後，計算平均損失和準確率
    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    if epoch == 3:
        torch.save(model.state_dict(), "distilbert_classifier_model_epoch_3.pth")

# 保存模型
torch.save(model.state_dict(), "distilbert_classifier_model_epoch_5.pth")


100%|██████████| 62979/62979 [36:18<00:00, 28.90it/s]


Epoch 1/5
Loss: 1.0975, Accuracy: 0.6032


100%|██████████| 62979/62979 [36:30<00:00, 28.76it/s]


Epoch 2/5
Loss: 0.9498, Accuracy: 0.6572


100%|██████████| 62979/62979 [36:30<00:00, 28.76it/s]


Epoch 3/5
Loss: 0.8409, Accuracy: 0.6970


100%|██████████| 62979/62979 [36:30<00:00, 28.75it/s]


Epoch 4/5
Loss: 0.7326, Accuracy: 0.7361


100%|██████████| 62979/62979 [36:33<00:00, 28.71it/s]


Epoch 5/5
Loss: 0.6289, Accuracy: 0.7739


In [20]:
model.load_state_dict(torch.load("distilbert_classifier_model_epoch_3.pth"))

<All keys matched successfully>

In [16]:
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'])
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [17]:
# input_ids = torch.tensor(tokenized_val['input_ids'].numpy()).to(device)
# attention_mask = torch.tensor(tokenized_val['attention_mask'].numpy()).to(device)
# token_type_ids = torch.tensor(tokenized_val['token_type_ids'].numpy()).to(device)
test_ids = torch.tensor(tokenized_test['input_ids'].numpy()).to(device)
test_attention_mask = torch.tensor(tokenized_test['attention_mask'].numpy()).to(device)

# val_input = {
#     'input_ids': input_ids,
#     'attention_mask': attention_mask,
#     # 'token_type_ids': token_type_ids,
# }

test_input = {
    'input_ids': test_ids,
    'attention_mask': test_attention_mask
}

# y_val = torch.tensor(y_val).to(device)

In [ ]:
pred = []
model.eval()  # 設置模型為評估模式
with torch.no_grad():  # 推理時不計算梯度
    for batch in tqdm(test_loader):
        input_ids, attention_mask = [x.to(device) for x in batch]
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(logits, dim=-1)
        predictions = torch.argmax(probabilities, dim=-1)
        class_labels = le.inverse_transform(predictions.cpu())
        pred.extend(class_labels)

 37%|███▋      | 9631/25749 [01:26<02:24, 111.85it/s]


KeyboardInterrupt: 

: 

In [19]:
submission['emotion'] = pred
submission.to_csv("submission_distilbert_epoch_5.csv", index=False)